# Simple Compound Device

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [0]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Make plots live-update while scans run.
from bluesky.utils import install_qt_kicker
install_qt_kicker()

from bluesky.examples import Syn2DGauss
from bluesky import RunEngine
from bluesky.callbacks import LiveTable, LiveGrid
from bluesky.plans import scan

RE = RunEngine()

## Data Acquisition

### Signals

This is a simple Signal. We can read it and set it.

In [0]:
from ophyd import Signal

x = Signal(value=0, name='x')

In [0]:
x.read()

In [0]:
x.set(3)

In [0]:
x.read()

The RunEngine calls these methods on ``x`` for us when we scan it:

In [0]:
RE(scan([], x, -1, 1, 3), LiveTable(['x']))

The ``Signal`` class is a "soft" signal not communicating with physical hardware. At beamlines, we more often use ``EpicsSignal`` which takes a PV string as its argument.

```python
x = EpicsSignal('PV:...', name ='x')
```

### Device

A Device is composed of Signals (or other Devices) that are bundled to create logical groups that map to our notion of a physical piece of hardware -- for example, an XY stage.

In [0]:
from ophyd import Device, Component as Cpt

class XYStage(Device):
    x = Cpt(Signal, value=0)
    y = Cpt(Signal, value=0)


stage = XYStage('', name='stage')

The readings from x and y are collated into one dictionary. The names of the components ('x' and 'y') are joined with the name of the overall device ('stage').

In [0]:
stage.read()

Each component signal can still be read or set individually.

In [0]:
stage.x.set(2)
stage.x.read()

In [0]:
stage.read()

### Built-in Devices

Ophyd includes some built-in Devices for EPICS. For example, ``EpicsMotor`` looks something like this:

```python
class EpicsMotor(Device):
    # position
    user_readback = Cpt(EpicsSignalRO, '.RBV')
    user_setpoint = Cpt(EpicsSignal, '.VAL', limits=True)

    # calibration dial <-> user
    user_offset = Cpt(EpicsSignal, '.OFF')
    user_offset_dir = Cpt(EpicsSignal, '.DIR')
    offset_freeze_switch = Cpt(EpicsSignal, '.FOFF')
    set_use_switch = Cpt(EpicsSignal, '.SET')

    # configuration
    velocity = Cpt(EpicsSignal, '.VELO')
    acceleration = Cpt(EpicsSignal, '.ACCL')
    motor_egu = Cpt(EpicsSignal, '.EGU')

    # motor status
    motor_is_moving = Cpt(EpicsSignalRO, '.MOVN')
    motor_done_move = Cpt(EpicsSignalRO, '.DMOV')
    high_limit_switch = Cpt(EpicsSignal, '.HLS')
    low_limit_switch = Cpt(EpicsSignal, '.LLS')
    direction_of_travel = Cpt(EpicsSignal, '.TDIR')

    # commands
    motor_stop = Cpt(EpicsSignal, '.STOP')
    home_forward = Cpt(EpicsSignal, '.HOMF')
    home_reverse = Cpt(EpicsSignal, '.HOMR')

    ...
```

Connecting to a specific motor looks like:

```python
motor = EpicsMotor('PV:...')
```

Likewise, connecting to an area detector is simple:

```python
ccd = Prosilica('PV:...')  # an area detector
```

### Using the custom XY Stage Device in a raster scan

In [0]:
from bluesky.plans import outer_product_scan, subs_wrapper

# Define a simulated detector that is coupled to our simulated XY stage.
det = Syn2DGauss('det',
                 stage.x, 'stage_x',
                 stage.y, 'stage_y',
                 center=(1, -1),
                 Imax=1000,
                 sigma=10,
                 noise='poisson')
det.exposure_time = 0.1

def raster_scan(x_start, x_stop, NX,
                y_start, y_stop, NY):
    "A variation on outer_product_scan that includes LiveGrid and LiveTable built in."
    fig, ax = plt.subplots()
    lg = LiveGrid((NY, NX), 'det',
                  xlabel='stage_x', ylabel='stage_y',
                  ax=ax,
                  extent=(x_start, x_stop,
                          y_start, y_stop))
    lt = LiveTable([det, stage])

    return (yield from subs_wrapper(
        outer_product_scan([det],
                  stage.y, y_start, y_stop, NY,
                  stage.x, x_start, x_stop, NX,
                  True),
        [lg, lt]))

In [0]:
RE(raster_scan(-1, 1, 3, -1, 1, 3))